In [1]:
using Pkg
Pkg.activate("rad_transfer"; shared=true)
using RadiativeTransfer
using RadiativeTransfer.CrossSection
using Pkg.Artifacts


 Activating environment at `~/.julia/environments/rad_transfer/Project.toml`


In [18]:
using NetCDF
using NCDatasets

In [12]:
reanalysis_file_path = "/export/data2/groupMembers/cchristo/reanalysis_3d/merra2/2015/01/MERRA2_400.tavg3_3d_asm_Nv.20150101.nc4"

"/export/data2/groupMembers/cchristo/reanalysis_3d/merra2/2015/01/MERRA2_400.tavg3_3d_asm_Nv.20150101.nc4"

In [62]:
# ds = ncread(reanalysis_file_path)
ds = Dataset(reanalysis_file_path)

times = ds["time"][:] 
lats = ds["lat"][:]
lons = ds["lon"][:]
T = ds["T"][:]
P = ds["PL"][:]

576×361×72×8 Array{Union{Missing, Float32},4}:
[:, :, 1, 1] =
 1.5  1.5  1.5  1.5  1.5  1.5  1.5  1.5  …  1.5  1.5  1.5  1.5  1.5  1.5  1.5
 1.5  1.5  1.5  1.5  1.5  1.5  1.5  1.5     1.5  1.5  1.5  1.5  1.5  1.5  1.5
 1.5  1.5  1.5  1.5  1.5  1.5  1.5  1.5     1.5  1.5  1.5  1.5  1.5  1.5  1.5
 1.5  1.5  1.5  1.5  1.5  1.5  1.5  1.5     1.5  1.5  1.5  1.5  1.5  1.5  1.5
 1.5  1.5  1.5  1.5  1.5  1.5  1.5  1.5     1.5  1.5  1.5  1.5  1.5  1.5  1.5
 1.5  1.5  1.5  1.5  1.5  1.5  1.5  1.5  …  1.5  1.5  1.5  1.5  1.5  1.5  1.5
 1.5  1.5  1.5  1.5  1.5  1.5  1.5  1.5     1.5  1.5  1.5  1.5  1.5  1.5  1.5
 1.5  1.5  1.5  1.5  1.5  1.5  1.5  1.5     1.5  1.5  1.5  1.5  1.5  1.5  1.5
 1.5  1.5  1.5  1.5  1.5  1.5  1.5  1.5     1.5  1.5  1.5  1.5  1.5  1.5  1.5
 1.5  1.5  1.5  1.5  1.5  1.5  1.5  1.5     1.5  1.5  1.5  1.5  1.5  1.5  1.5
 1.5  1.5  1.5  1.5  1.5  1.5  1.5  1.5  …  1.5  1.5  1.5  1.5  1.5  1.5  1.5
 1.5  1.5  1.5  1.5  1.5  1.5  1.5  1.5     1.5  1.5  1.5  1.5  1.5  1.5  1.5
 1

In [54]:
# ncinfo(reanalysis_file_path)
desired_lat = 72.5796
desired_lon = -38.4588

# find index of nearest gridcell
lat_ind = argmin(broadcast(abs, lats .- desired_lat))
lon_ind = argmin(broadcast(abs, lons .- desired_lon))

227

In [31]:
ds["T"]

576×361×72×8 NCDatasets.CFVariable{Union{Missing, Float32},4,NCDatasets.Variable{Float32,4,NCDataset},NCDatasets.Attributes{NCDataset{Nothing}},NamedTuple{(:fillvalue, :scale_factor, :add_offset, :calendar, :time_origin, :time_factor),Tuple{Float32,Float32,Float32,Nothing,Nothing,Nothing}}}:
[:, :, 1, 1] =
 173.846  173.689  173.447  173.135  …  201.814  202.127  202.492  202.914
 173.846  173.689  173.447  173.127     201.828  202.137  202.496  202.914
 173.846  173.682  173.439  173.127     201.844  202.146  202.5    202.914
 173.846  173.682  173.439  173.119     201.859  202.156  202.506  202.914
 173.846  173.682  173.432  173.119     201.875  202.166  202.512  202.914
 173.846  173.682  173.432  173.111  …  201.893  202.176  202.516  202.914
 173.846  173.682  173.432  173.111     201.908  202.187  202.521  202.914
 173.846  173.674  173.424  173.111     201.924  202.197  202.525  202.914
 173.846  173.674  173.424  173.103     201.939  202.207  202.531  202.914
 173.846  173.674

In [5]:
# fetch('H2O_S',1,1,xmin,xmax)
# fetch('CO2_S',2,1,xmin,xmax)
# fetch('CH4_S',6,1,xmin,xmax)

## Open Profile

In [ ]:
hitran_data_CO2 = read_hitran("hitran_molec_id_2_CO2.tar", mol=2, iso=1, ν_min=400, ν_max=2100)
hitran_data_H2O = read_hitran("hitran_molec_id_1_H2O.tar", mol=1, iso=1, ν_min=400, ν_max=2100)
hitran_data_CH4 = read_hitran("hitran_molec_id_6_CH4.tar", mol=6, iso=1, ν_min=400, ν_max=2100)

In [3]:
hitran_data_CO2

UndefVarError: UndefVarError: hitran_data_CO2 not defined

In [3]:
model_voigt_GPU = make_hitran_model(hitran_data, Voigt(), architecture=Architectures.GPU())


UndefVarError: UndefVarError: hitran_data not defined